<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing an Agno Policy Research Agent</h1>

In [ ]:
!pip install -q agno duckduckgo-search newspaper4k lxml_html_clean openinference-instrumentation-agno openinference-instrumentation-openai openai arize-phoenix

## Connect to Phoenix

For this example you'll use a hosted version of Phoenix - if you don't already have an account, you can create one for free [here](https://app.phoenix.arize.com/). If you'd prefer to self-host Phoenix, then follow the [instructions here](https://arize.com/docs/phoenix/self-hosting).

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("🔑 Enter your Phoenix API key: ")

if "PHOENIX_COLLECTOR_ENDPOINT" not in os.environ:
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = getpass("🔑 Enter your Phoenix Collector Endpoint")

In [ ]:
from openinference.instrumentation.agno import AgnoInstrumentor
from openinference.instrumentation.openai import OpenAIInstrumentor

from phoenix.otel import register

tracer_provider = register(project_name="agno")
AgnoInstrumentor().instrument(tracer_provider=tracer_provider)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

## Building your Agno Agent

In [ ]:
from textwrap import dedent

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.newspaper4k import Newspaper4kTools

policy_research_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    tools=[DuckDuckGoTools(), Newspaper4kTools()],
    description=dedent("""\
        You are a senior policy analyst who advises G20 governments on the
        governance of advanced AI systems. Your expertise spans: 🌐

        • Comparative regulatory analysis (EU AI Act, U.S. EO, China’s draft regs)
        • Risk taxonomy & mitigation frameworks (NIST, ISO, OECD)
        • Multi‑stakeholder negotiation and diplomacy
        • Economic impact modelling and labour‑market studies
        • Standards‑setting processes (IEEE, ISO/IEC)
        • Enforcement mechanisms and audit requirements
        • Rights‑based and ethics‑based approaches to AI governance\
    """),
    instructions=dedent("""\
        1. Discovery Phase 🔍
           – Gather at least 12 authoritative sources: legislation, white‑papers,
             peer‑reviewed studies, think‑tank reports, and reputable news.
           – Prioritise the most recent versions / amendments (≤ 12 months).
           – Identify divergent regional approaches and key stakeholders.

        2. Comparative Analysis 📊
           – Map each region’s regulatory scope, risk tiers, and enforcement powers.
           – Cross‑reference impact assessments and economic forecasts.
           – Highlight areas of convergence and friction (e.g., foundation‑model audits).

        3. Recommendation Draft ✍️
           – Craft a concise, actionable brief for policymakers.
           – Include trade‑offs, implementation timelines, and anticipated market effects.
           – Use bullet points and tables where clarity improves.

        4. Validation & Quality Control ✓
           – Verify every cited statute / article for publication date and authenticity.
           – Ensure balanced representation of civil‑society and industry viewpoints.
           – Flag any major uncertainties or data gaps.
    """),
    expected_output=dedent("""\
        # {Short, Punchy Headline on AI Governance Landscape} 🌐

        ## Executive Summary
        {One‑paragraph snapshot of regulatory momentum and stakes}

        | Region | Current Status | Key Provisions | Enforcement Timeline |
        |--------|---------------|----------------|----------------------|
        | EU     | ...           | ...            | ...                 |
        | U.S.   | ...           | ...            | ...                 |
        | ...    | ...           | ...            | ...                 |

        ## Comparative Findings
        - **Risk Classification:** {...}
        - **Testing & Audit Requirements:** {...}
        - **Penalties & Incentives:** {...}

        ## Strategic Implications
        {Market, innovation, and compliance impacts for enterprises}

        ## Policy Recommendations
        1. **Short‑Term (0‑12 mo):** {...}
        2. **Medium‑Term (1‑3 yrs):** {...}
        3. **Long‑Term (>3 yrs):** {...}

        ## Sources
        {Numbered list, each with publication date and 1‑line relevance note}

        ---
        Prepared by AI Policy Analyst · Published: {current_date} · Last Updated: {current_time}
    """),
    markdown=True,
    show_tool_calls=True,
    add_datetime_to_instructions=True,
)

In [ ]:
policy_research_agent.print_response(
    "Analyze the current state and future implications of artificial intelligence regulation worldwide",
    stream=True,
)

## View your Traces in Phoenix!

![traces in phoenix](https://storage.googleapis.com/arize-phoenix-assets/assets/images/agno-example-trace.png)